In [1]:
import pandas as pd
import numpy as np

In [6]:
class DataSourcing:
  def __init__(self):
    pass
  
  def open_csv(self, path):
    df = pd.read_csv(path)
    return df
  
  def give_info(self, df):
    message =  f"""
    <------------------------------------------------------------------------DESCRIPTION OF THE DATAFRAME IN QUESTION!!!----------------------------------------------------------------------->
    There are {df.shape[0]} rows and {df.shape[1]} columns in the dataframe.
    There are {len(df.columns)} columns, namely: {df.columns}.  
    ------------------------------------------------------------------------------------------------------------------------->
    
    
    The first 5 records in the df are seen here:
    ------------------------------------------------------------------------------------------------------------------------->
    {df.head()}
    ------------------------------------------------------------------------------------------------------------------------->
    
    
    The last 5 records in the df are as follows: 
    ------------------------------------------------------------------------------------------------------------------------->
    {df.tail()}
    ------------------------------------------------------------------------------------------------------------------------->
    
    
    The descriptive statistics of the df (mean,median, max, min, std) are as follows:
    ------------------------------------------------------------------------------------------------------------------------->
    {df.describe()}
    ------------------------------------------------------------------------------------------------------------------------->
    """
    print (message)
  

In [7]:
class DataPreProcessing(DataSourcing):
  def dropColumns(self, df, columns):
    df = df.drop(columns, axis=1)
    return df

  def dropRows(self, df, rows):
    df = df.drop(rows, axis=0)
    return df

In [17]:
data = DataSourcing()
df = data.open_csv('bigml_59c28831336c6604c800002a.csv')
data.give_info(df)


    <------------------------------------------------------------------------DESCRIPTION OF THE DATAFRAME IN QUESTION!!!----------------------------------------------------------------------->
    There are 3333 rows and 21 columns in the dataframe.
    There are 21 columns, namely: Index(['state', 'account length', 'area code', 'phone number',
       'international plan', 'voice mail plan', 'number vmail messages',
       'total day minutes', 'total day calls', 'total day charge',
       'total eve minutes', 'total eve calls', 'total eve charge',
       'total night minutes', 'total night calls', 'total night charge',
       'total intl minutes', 'total intl calls', 'total intl charge',
       'customer service calls', 'churn'],
      dtype='object').  
    ------------------------------------------------------------------------------------------------------------------------->
    
    
    The first 5 records in the df are seen here:
    --------------------------------------------